## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd #импортируем pandas
paying_capacity = pd.read_csv('datasets/data.csv')
print('Общая информация о данных:')
print()
paying_capacity.info()  #получаем общую информацию о данных
print()
print('Первые 10 строк данных:')
paying_capacity.head(10)

Общая информация о данных:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB

Первые 10 строк данных:


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Полученные данные имеют отрицательные значения в столбце трудового стажа, также значения разного порядка. Возможно, в отдельных случаях стаж указан в часах (наблюдается у пенсионеров). Также имеются артефакты в графе образования - значения указаны в разных регистрах. Цели кредита сформулированы по разному, и для анализа потребуется создание словаря. Исходя из общей информации о полученных данных, в столбцах стажа работы и полученного дохода имеются пропущенные значения.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
def change_nan_to_median(data, income_type, searching_column, type_massive): #создадим функцию для замены NaN
                                                #на медианные значения по профессиям, в необходимых столбцах
    for type_name in type_massive:
        rule1 = data[income_type] == type_name  #задаем правило фильтрации по профессиям
        median_search = data.loc[rule1, searching_column].median()  #ищем медианное значение
        data.loc[rule1, searching_column] = data.loc[rule1, searching_column].fillna(value = median_search)  #заполняем
                                                                                                    #пропущенные значения

type_massive = paying_capacity['income_type'].unique()   #создаем список профессий

change_nan_to_median(paying_capacity, 'income_type', 'days_employed', type_massive) #применяем функцию change_nan_to_median
                                                           #для замены пропущенных значений в столбце общего трудового стажа
change_nan_to_median(paying_capacity, 'income_type', 'total_income', type_massive)  #применяем функцию change_nan_to_median
                                                           #для замены пропущенных значений в столбце общего дохода
print('Общая информация о данных:')
print()
print(paying_capacity.info())

Общая информация о данных:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


### Вывод

Были пропущены даные в столбцах с трудовым стажем и ежемесячным доходом. Причем пропущенны данные были также и у работающих людей, поэтому можно предположить, что по какой-то причине при выгрузке данные были пропущены. Чтобы заполнить пропущенные данные, были найдены медианные значения трудового стажа для каждой группы профессий.

### Устранение несоответствий в данных

In [3]:
print('Категории людей с положительным трудовым стажем:')
print(paying_capacity[paying_capacity['days_employed'] > 0]['income_type'].unique()) #проверяем, у кого есть положительный
                                                                                     #трудовой стаж
print()
print('Безработные с положительным трудовым стажем:')
paying_capacity[(paying_capacity['days_employed'] > 0) & (paying_capacity['income_type'] == 'безработный')] #проверяем, есть
                                                          #ли ошибки в указанных данных у безработных с положительным стажем

Категории людей с положительным трудовым стажем:
['пенсионер' 'безработный']

Безработные с положительным трудовым стажем:


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,337524.466835,31,среднее,1,женат / замужем,0,M,безработный,1,59956.991984,покупка жилья для сдачи
14798,0,395302.838654,45,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью


Примем , что все значения больше 0 скопировались неверно и переконвертировались из дней в часы. Очевидно, что у безработных есть несоответствие в и возрасте и количестве дней трудового стажа. Т.к. строчек с такой аномалией всего 2 на весь датасет - можно их удалить.

In [4]:
paying_capacity = paying_capacity.where(paying_capacity['income_type'] != 'безработный').dropna().reset_index(drop=True)
#удаляем все значения "безработный"

Проверим на нулевые значения возраста клиента в годах

In [5]:
print('Количество строк с нулевым значением возраста:', end=' ')
print(paying_capacity[paying_capacity['dob_years'] == 0]['dob_years'].count())  #выводим количество строк с нулевым
                                                                                #значением возраста

Количество строк с нулевым значением возраста: 101


Так как количество нулевых значений возраста клиента незначительно, можно удалить эти аномалии.

In [6]:
paying_capacity = paying_capacity.where(paying_capacity['dob_years'] != 0).dropna().reset_index(drop=True)

Проверим, правильны ли данные по трудовому стажу отноительно возраста заявителя. Согласно Трудовому Кодексу РФ, работать разрешено с 16 лет, соответственно данные заявителей, чей возраст начала работы менее 16 лет, являются аномалией. Такая аномалия, возможна из-за неправильно посчитанного трудового стажа.

In [7]:
#проверяем соответствие возраста общему трудовому стажу
#принимаем, что работать разрешено с 16 лет (согласно ТК РФ)
rule2 = paying_capacity['days_employed'] > 0 #правило для поиска значений больше 0
paying_capacity.loc[rule2, 'days_employed'] = paying_capacity.loc[rule2, 'days_employed'] / 24 #преобразуем часы в дни
paying_capacity['days_employed'] = paying_capacity['days_employed'].abs()
paying_capacity['age_check'] = paying_capacity['dob_years'] - (paying_capacity['days_employed'] / 365.25)

print('Количество людей, у которых стаж работы идет с возраста меньше 16 лет:', end=' ')
print(paying_capacity[(paying_capacity['age_check'] < 16)]['age_check'].count())

Количество людей, у которых стаж работы идет с возраста меньше 16 лет: 1529


Всего аномалия с возрастом начала трудового стажа наблюдается у значительного количества клиентов. Уменьшим их рабочий стаж, чтобы он удовлетворял минимально разрешенным требованиям по возрасту.

In [8]:
rule3 = paying_capacity['age_check'] < 16  #задаем правило для индексации
check_days_employed_value = (16.01 - paying_capacity.loc[rule3, 'age_check']) * 365.25  #величина, на которую будем
                                                                                    #корректировать значение days_employed
paying_capacity.loc[rule3, 'days_employed'] = paying_capacity.loc[rule3, 'days_employed'] - check_days_employed_value   
                                                            #производим корректировку аномальных значений days_employed
paying_capacity = paying_capacity.loc[:, 'children': 'purpose'] #удаляем столбец age_check

Проверим на аномалии значение пола

In [9]:
print('Уникальные значения столбца gender:')
print(paying_capacity['gender'].value_counts()) #считаем уникальные значения столбца gender

Уникальные значения столбца gender:
F      14163
M       7258
XNA        1
Name: gender, dtype: int64


Удаляем аномалии

In [10]:
paying_capacity = paying_capacity.where(paying_capacity['gender'] != 'XNA').dropna().reset_index(drop=True)
#удаляем строку со значением пола "XNA"

Проверим на аномалии количество детей.

In [11]:
paying_capacity['children'].value_counts()

 0.0     14078
 1.0      4801
 2.0      2042
 3.0       328
 20.0       75
-1.0        47
 4.0        41
 5.0         9
Name: children, dtype: int64

Удаляем строки с отрицательным количеством детей и значением 20.

In [12]:
paying_capacity = paying_capacity.where(paying_capacity['children'] != -1).dropna().reset_index(drop=True)
paying_capacity = paying_capacity.where(paying_capacity['children'] != 20).dropna().reset_index(drop=True)

### Вывод
Аномалии были выявлены и устранены. Возможно, они появились в процессе обработки, внесения данных. Возможно кто-то из клиентов просто не указал требуемые данные.

### Замена типа данных

In [13]:
print('Типы данных в датасете:')
print()
paying_capacity.info()  #проверяем типы данных в датасете до обработки

Типы данных в датасете:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21299 entries, 0 to 21298
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21299 non-null  float64
 1   days_employed     21299 non-null  float64
 2   dob_years         21299 non-null  float64
 3   education         21299 non-null  object 
 4   education_id      21299 non-null  float64
 5   family_status     21299 non-null  object 
 6   family_status_id  21299 non-null  float64
 7   gender            21299 non-null  object 
 8   income_type       21299 non-null  object 
 9   debt              21299 non-null  float64
 10  total_income      21299 non-null  float64
 11  purpose           21299 non-null  object 
dtypes: float64(7), object(5)
memory usage: 2.0+ MB


Проведем замену типов данных в датасете.

In [14]:
def change_mean_type(data, column_name, final_type):   #функция замены типа данных в столбцах: data - датасет, column_name - 
                          #названия столбцов, final_type - тип данных, на который необходимо поменять текущий тип
    for name in column_name:
        data[name] = data[name].astype(final_type, errors='ignore')  #меняем тип данных, ошибки игнорируем (оставляем
                                                        #оригинальные значения - если значения в столбце не числовые)
        
change_mean_type(paying_capacity, paying_capacity.columns, 'int')  #применяем функцию: датасет - paying_capacity
                        #названия столбцов - paying_capacity.columns, конечный тип данных - int64 (целочисленный)
    
print('Типы данных в датасете после замены:')
print()
paying_capacity.info()  #проверяем типы данных в датасете после замены

Типы данных в датасете после замены:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21299 entries, 0 to 21298
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21299 non-null  int64 
 1   days_employed     21299 non-null  int64 
 2   dob_years         21299 non-null  int64 
 3   education         21299 non-null  object
 4   education_id      21299 non-null  int64 
 5   family_status     21299 non-null  object
 6   family_status_id  21299 non-null  int64 
 7   gender            21299 non-null  object
 8   income_type       21299 non-null  object
 9   debt              21299 non-null  int64 
 10  total_income      21299 non-null  int64 
 11  purpose           21299 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Значения с плавающей точкой были заменены на целочисленные значения. Был выбран метод .astype(), т.к. исходный тип данных был числом с плавающей точкой, а все пропуски были устранены ранее.

### Обработка дубликатов

В столбце education образование клиентов указаны в разных регистрах, поэтому для корректного анализа необходимо привести все значения к нижнему регистру.

In [15]:
paying_capacity['education'] = paying_capacity['education'].str.lower()  #сохраняем значения образования в нижнем регистре

print('Количество строк, имеющих дубликаты:')
print()
print(paying_capacity[paying_capacity.duplicated()].count()) #проверяем количество дублирующихся строк

Количество строк, имеющих дубликаты:

children            71
days_employed       71
dob_years           71
education           71
education_id        71
family_status       71
family_status_id    71
gender              71
income_type         71
debt                71
total_income        71
purpose             71
dtype: int64


Удалим из датасета дубликаты.

In [16]:
paying_capacity = paying_capacity.drop_duplicates(keep=False).reset_index(drop=True)  #удаляем дублирующиеся строки
print('Количество строк, имеющих дубликаты после удаления:')
print()
print(paying_capacity[paying_capacity.duplicated()].count()) #проверяем количество дублирующихся строк после удаления

Количество строк, имеющих дубликаты после удаления:

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


### Вывод

В столбце education образование клиентов были указаны в разных регистрах. Скорее всего, это артефакты выгрузки. Для исключения разного написанияодних и тех же значений, в целях проведения последующего анализа, перевели все значения в нижний регистр.
Проверили датасет на дублирующиеся строк и удалили их.

### Лемматизация

In [17]:
from pymystem3 import Mystem #импортируем библиотеку pymystem3 для проведения лемматизации
m = Mystem()

purpose_lemmas = []   #создаем пустой набор
for text_value in paying_capacity['purpose']:
    append_text = m.lemmatize(text_value)
    purpose_lemmas.append(append_text)  #записываем данные

paying_capacity['purpose_lemmas'] = purpose_lemmas  #создаем новый столбец с выделенными леммами

purpose_lemmas_cut = []  #создаем пустой набор для выделения отдельных слов из выделенных лемм
for string_element in purpose_lemmas:
    for element in string_element:
        purpose_lemmas_cut.append(element) #записываем слова

from collections import Counter  #подключаем счетчик
print(Counter(purpose_lemmas_cut))  #просматриваем самые часто встречающиеся цели кредита

Counter({' ': 33140, '\n': 21162, 'недвижимость': 6274, 'покупка': 5822, 'жилье': 4400, 'автомобиль': 4249, 'образование': 3961, 'с': 2880, 'операция': 2570, 'свадьба': 2278, 'свой': 2207, 'на': 2187, 'строительство': 1859, 'высокий': 1358, 'получение': 1302, 'коммерческий': 1294, 'для': 1277, 'жилой': 1213, 'сделка': 930, 'заниматься': 896, 'дополнительный': 892, 'проведение': 752, 'сыграть': 747, 'сдача': 644, 'семья': 633, 'собственный': 626, 'со': 620, 'ремонт': 598, 'подержанный': 476, 'подержать': 470, 'приобретение': 456, 'профильный': 431})


In [18]:
lemmas_categories = ['недвижимость', 'автомобиль', 'образование', 'свадьба']  #формируем список целей кредита

def lemmas_category(row):   #функция для выделения категорий из полученных лемм
    for word in lemmas_categories:
        if word in row:
            return word
        elif 'ремонт' in row and 'жилье' in row:
            return 'ремонт'
        elif 'жилье' in row:   #формируем правило 'жилье' == 'недвижимость'
            return 'недвижимость'

    
paying_capacity['purpose_categories'] = paying_capacity['purpose_lemmas'].apply(lemmas_category)   #применяем функцию для
                                            #столбца 'purpose_lemmas' и создаем новый столбец с категориями целей кредита

paying_capacity = paying_capacity.loc[:, ['children', 'days_employed', 'dob_years', 'education', 'education_id',
                 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose', 
                 'purpose_categories']]   #удаляем столбец 'purpose_lemmas'

paying_capacity.loc[:, 'family_status': 'purpose_categories'].head() #проверяем применение категорий

,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_categories
0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


### Вывод

Были выделены леммы из столюца с целями получения кредита. Была использована библиотека pymystem3, так как необходимо было получить именно слова, а не их составные части. Проанализированы наиболее часто встречающиеся причины подачи заявления на кредит, составлен список категорий целей. На основе выделенных лемм был сформирован новый столбец с категориями целей получения кредитов. 

### Категоризация данных

Для облегчения анализа создадим словарь категорий для образования.

In [19]:
paying_capacity_education_dict = paying_capacity[['education_id', 'education']].drop_duplicates().reset_index(drop=True)
                                                                        #создадим словарь ID для категории "образование"
paying_capacity_education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Точно так же создадим словарь для семейного положения.

In [20]:
paying_capacity_family_status_dict = paying_capacity[['family_status_id', 
                                                      'family_status']].drop_duplicates().reset_index(drop=True)
                                                            #создадим словарь ID для категории "семейное положение"
paying_capacity_family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [21]:
print('Получим квантили по зарплате:', '\n', paying_capacity['total_income'].quantile([0.25, 0.50, 0.75]))  #найдем квантили

def income_categorising(row):  #функция, возвращающая категории дохода
    if row < 107425:
        return 'низкий доход'
    if 107425 <= row < 142594:
        return 'средний доход'
    if 142594 <= row < 196049:
        return 'выше среднего доход'
    else:
        return 'высокий доход'
    
paying_capacity['income_categorised'] = paying_capacity['total_income'].apply(income_categorising)
paying_capacity.head()

Получим квантили по зарплате: 
 0.25    107425.75
0.50    142594.00
0.75    196049.00
Name: total_income, dtype: float64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_categories,income_categorised
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,высокий доход
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,средний доход
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,выше среднего доход
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высокий доход
4,0,13510,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,выше среднего доход


Отредактируем исходную таблицу - удалим словесные описания уровня образования и семейного положения. Также удалим столбцы, ненужные для ответа на поставленные вопросы.

In [22]:
paying_capacity_checked = paying_capacity.loc[:, ['children', 'education_id', 'family_status_id', 'debt', 'total_income', 
                                          'purpose_categories', 'income_categorised']]
paying_capacity_checked.head()

,children,education_id,family_status_id,debt,total_income,purpose_categories,income_categorised
0,1,0,0,0,253875,недвижимость,высокий доход
1,1,1,0,0,112080,автомобиль,средний доход
2,0,1,0,0,145885,недвижимость,выше среднего доход
3,3,1,0,0,267628,образование,высокий доход
4,0,1,1,0,158616,свадьба,выше среднего доход


### Вывод

Были созданы словари для категоризации столбцов "образование" и "семейное положение".В отредактированной таблице были оставлены только столбцы с ID данных категорий. Также категоризирован общий доход.  Были удалены столбцы, не требующиеся для ответа на поставленные ниже вопросы.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [23]:
children_debt_pivot = paying_capacity_checked.pivot_table(index='children', columns='debt', 
                                                          aggfunc={'debt':'count'}).fillna(0).astype(int)  #формируем
                                                                                                    #сводную таблицу
def find_and_write_percentage(data):  #функция нахождения процентов просрочки кредита от общего числа кредитов
    data['percentage'] = ((data[('debt', 1)] * 100) / (data[('debt', 0)] + data[('debt', 1)])).round(2)

find_and_write_percentage(children_debt_pivot)  #находим процент просроченных кредитов для каждого количества детей
children_debt_pivot.sort_values(by='percentage', ascending=False)

debt       percentage
debt          0     1           
children                        
4            37     4       9.76
2          1842   194       9.53
1          4341   440       9.20
3           301    27       8.23
0         12909  1058       7.57
5             9     0       0.00

In [24]:
def percentage_of_debt_kids(kids, column_no_debt, column_debt):  #функция нахождения процентного соотношения отданных
    #кредитов в зависимости от наличия детей: kids - дети есть/нет (0/1), column_no_debt - столбец "без долгов",
                                                                         #column_debt - столбец "с долгами"
        
    all_no_debt = children_debt_pivot.loc[kids:, column_no_debt].sum()  #сумма всех заявителей без долгов с детьми
    all_debt = children_debt_pivot.loc[kids:, column_debt].sum()  #сумма всех заявителей с долгами
    total_with_no_debt = all_no_debt + all_debt  #сумма всех заявителей с детьми
    percentage_debt = all_debt / total_with_no_debt  #расчет процента отданных кредитов с детьми
    return percentage_debt

print('Процент просроченных кредитов заявителями с детьми:  {:.2%}'.format(percentage_of_debt_kids(1, ('debt', 0), ('debt', 1))))
print('Процент просроченных кредитов заявителями без детей: {:.2%}'.format(percentage_of_debt_kids(0, ('debt', 0), ('debt', 1))))

Процент просроченных кредитов заявителями с детьми:  9.24%
Процент просроченных кредитов заявителями без детей: 8.14%


### Вывод

Процент просрочки у заявителей с 5 детьми равен 0, но стоит отметить, что таких заявителей всего 9, поэтому считаю выборку нерепрезентативной. Чтобы делать выводы по этой категории заявителей, следует накопить больше данных.
Наибольший процент просроченных кредитов у семей с 4 детьми, однако заявителей с таким количеством детей в разы меньше, чем клиентов с числом детей от 1 до 3, поэтому возможно, принакоплении большей информации, статистика изменится.
Далее следуют клиенты с 2 детьми. Такие клиенты почти на 2% реже отдают кредиты, чем заявители без детей. Чаще на 0,3% кредиты отдают семьи с 1 ребенком. Процент просрочки у клиентов с 3 детьми самый маленький: на 1.3% ниже, чем у заявителей с 2 детьми. Процент просроченных кредитов у заявителей без детей самый маленький.
В целом, процент просроченных кредитов у заявителей без детей меньше, чем у заявителей с детьми примерно на 1%. Также, исходя из полученных данных, они чаще подают заявки на кредит. С учетом того, что количество заявителей без детей примерно в 2 раза больше, процентное различие можно считать незначительным.
Можно сделать вывод, что зависимости между наличием детей и возвратом кредита в срок нет.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [25]:
family_status_id_pivot = paying_capacity_checked.pivot_table(
    index='family_status_id', columns='debt', aggfunc={'debt':'count'}).merge(
    paying_capacity_family_status_dict, on='family_status_id', how='left')  #формируем сводную таблицу и для удобства
                                                                #возвращаем текстовое написание семейного положения
    
find_and_write_percentage(family_status_id_pivot)
family_status_id_pivot.sort_values(by='percentage', ascending=False)

/Users/antonkirillov/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,family_status_id,"(debt, 0)","(debt, 1)",family_status,percentage
4,4,2505,272,Не женат / не замужем,9.79
1,1,3705,383,гражданский брак,9.37
0,0,11251,922,женат / замужем,7.57
3,3,1095,84,в разводе,7.12
2,2,883,62,вдовец / вдова,6.56


### Вывод

Процент просрочки кредита примерно одинаков, однако выделяются вдовцы - их процент возврата самый низкий, затем идут женатые или в разводе. Самые безответственные - неженатые и живущие в гражданском браке. Стоит отметить, что выборка по женатым заявителям в разы больше остальных категорий.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [26]:
#формируем сводную таблицу
income_pivot = paying_capacity_checked.pivot_table(index='income_categorised', 
                                                   columns='debt', 
                                                   aggfunc={'debt':'count'})
                                                                                              

find_and_write_percentage(income_pivot)
income_pivot.sort_values(by='percentage', ascending=False)

debt      percentage
debt                    0    1           
income_categorised                       
средний доход        3973  381       8.75
выше среднего доход  5687  540       8.67
низкий доход         4867  423       8.00
высокий доход        4912  379       7.16

### Вывод

Процент просрочки кредита для разных уровней дохода примерно одинаков. Небольшая зависимость наблюдается. Например, чаще всего отдают кредиты люди с высоким доходом, затем идут люди с низким доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [27]:
purpose_pivot = paying_capacity_checked.pivot_table(index='purpose_categories', columns='debt', aggfunc={'debt':'count'})
                                                                                              #формируем сводную таблицу
find_and_write_percentage(purpose_pivot)
purpose_pivot.sort_values(by='percentage', ascending=False)

debt      percentage
debt                   0    1           
purpose_categories                      
автомобиль          3852  397       9.34
образование         3592  369       9.32
свадьба             2097  181       7.95
недвижимость        9335  741       7.35
ремонт               563   35       5.85

### Вывод

Наименьший процент просроченных кредитов наблюдается у заявителей, взявших кредит на ремонт. Затем примерно одинаковый процент просрочки кредита на свадьбу и покупку недвижимости. Процент просроченных кредитов на автомобиль и образование самый большой. Стоит отметить, что кредиты на ремонт берут почти в 10 раз меньше, чем на недвижимость, поэтому, возможно, когда выборка по ремонту будет больше, результат изменится.

### Шаг 4. Общий вывод

Данные были получены с аномалиями, которые в процессе обработки были успешно устранены. 
При проведении анализа были категоризированны данные, созданы словари.

Получены ответы на поставленные вопросы.

Наблюдается различие между отданными кредитами людей без детей и с детьми в 1% в пользу бездетных. 
Количество клиентов с детьми в 2 раза меньше, поэтому данное различие можно считать незначительным. Можно сделать вывод, что наличие детей в целом не влияет на вероятность просрочки. 
Стоит отметить, что наблюдаются зависимости между количеством детей и вероятностью просрочки кредита. Например, заявители с 3 детьми на 1.5% реже имеют просрочку по кредиту по сравнению с клиентами с 4 детьми. Вероятность просрочки кредита заявителями с 1-2 детьми примерно одинакова.

Процент просрочки кредита ( в зависимости от семейного положения) примерно одинаков, однако выделяются вдовцы - их процент меньшее, чем у неженатых, примерно на 3%. Затем идут женатые или в разводе, они отдают кредиты реже вдовцов примерно на 1%. Самые безответственные - неженатые и живущие в гражданском браке. Выборка по женатым заявителям в разы больше остальных категорий.

Наблюдается небольшая зависимость между уровнем дохода и вероятностью просрочки кредита. Например, реже всего допускают просрочку кредита люди с высоким доходом, затем идут люди с низким доходом. Клиенты со средним доходим отдают кредиты на 1,5% реже клиентов с высоким доходом.

Процент просроченных кредитов заявителей, бравших кредит на ремонт, ниже, чем у берущих кредит на свадьбу или покупку недвижимости. Кредиты на автомобиль и образование возвращаются реже всего. Возможно, результат изменится, когда выборка по людям, берущим кредит на ремонт станет больше - на данный момент количество таких клиентов почти в 20 раз меньше взявших кредит на недвижимость.